# Sentiment Analysis

# Data : 네이버 영화 리뷰

In [101]:
import numpy as np
import pandas as pd

In [102]:
train_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [103]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [104]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [105]:
len(train_data), len(test_data)

(150000, 50000)

# Q1. 트레이닝 데이터를 트레이닝 셋과 벨리데이션 셋으로 분할하시오.

**규칙**
1. train : valid = 8 : 2
2. label이 골고루 분할되어 들어가도록 stratify하게 분할한다.
3. random state, seed 등은 2021로 통일
4. 변수명 :
    * x_train, x_valid
    * y_train, y_valid
5. test_data도, x와 y를 분리해둔다.
    * x_test, y_test

In [106]:
train_data.tail()

,id,document,label
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [107]:
####################
## Your Code here ##
####################
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_data, test_size = 0.2, random_state = 2021)
train_data.reset_index(drop = True, inplace = True)
val_data.reset_index(drop = True, inplace = True)

In [108]:
train_data.shape, val_data.shape, test_data.shape

((120000, 3), (30000, 3), (50000, 3))

# Q2. x를 문장들이 담긴 리스트로 변환하시오.

ex> ['와 이 영화 재밌음', '크아앙 울부짖었다' ,... ]

In [109]:
## 위의 한국어 전처리를 했다면 사용하지 않는다.
# 명확히 str로 데이터 타입을 변경해 줘야 tokenizer에서 에러가 없다. 한국어 건들 때만 그러는 것 같음.
x_train = train_data['document'].astype('str').tolist()
x_val = val_data['document'].astype('str').tolist()
x_test = test_data['document'].astype('str').tolist()

In [110]:
x_train[:5]

['졸작이다.. 돈이 아깝다',
 '정치인에게 가장 중요한 말은? 꼬리 잡히지 말것.',
 '와나 진짜너무하네ㅡㅡ 머고이게 영화가?',
 '내용도 철학도 재미도 아무것도 없는 그냥 OOO기 귀신영화..흥신소직원 연기도 엉망이고. 연기잘한다는 소리 듣고싶어서 힘이 들어간게 보일정도..근10년동안 이렇게 한심한 영화는 처음입니다. 시간낭비 하지마시길',
 '오즈의 영역을 넘어 마술의 경지에 이르는 허우.']

In [111]:
# label로 넘긴다
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Q3. X를 tokenize하고, index의 sequence로 바꾸어 내시오.

**규칙**
1. 상위 40000개 단어만 사용하기로 한다.

In [112]:
####################
## Your Code here ##
####################
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 40000 # 상위 사만개의 단어만 보존
tokenizer = Tokenizer(num_words = max_words)

In [113]:
# Tokenizer
%%time
# tokenizer token : idx 토크나이즈하고 인덱스 다 붙이는 것. 과정이 매칭되어야 한다.
tokenizer.fit_on_texts(x_train)
# Text --> Sequence, text seq > idx seq 매칭
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)
x_test = tokenizer.texts_to_sequences(x_test)

CPU times: user 7.53 s, sys: 76.9 ms, total: 7.61 s
Wall time: 7.91 s


# Q4. 문장의 길이를 통일하시오.

**규칙**
1. pad는 문장의 앞에 덧댈 것.
2. 문장 최대 길이는 40


In [125]:
####################
## Your Code here ##
####################

# 위에서 설정한 사만
max_words = max_words
# 단어 embedding 차원
embedding_dim = 192
# 문장 최대 길이
max_len = 40

In [115]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [116]:
### Pad Sequences > 문장 길이 통일
x_train = pad_sequences(x_train, maxlen = max_len)
x_val = pad_sequences(x_val, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [117]:
x_train[3]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   128, 36110,   147,   493,    26,     6,
         936, 20658,   116,  3938, 36111,  1688,  1318, 36112,    34,
        1319,    18,  4899,  1080], dtype=int32)

In [118]:
train_data.iloc[3]

id                                                    7275627
document    내용도 철학도 재미도 아무것도 없는 그냥 OOO기 귀신영화..흥신소직원 연기도 엉망...
label                                                       0
Name: 3, dtype: object

In [119]:
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

In [120]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((120000, 40), (30000, 40), (50000, 40), (120000,), (30000,), (50000,))

# Q5. X에서, 0으로만 데이터를 제거하시오.

In [121]:
####################
## Your Code here ##
####################
# 그 값이 0인 행들은 False 아니면 1로 만든다.
not0train_idx = x_train.sum(1) > 0
not0val_idx = x_val.sum(1) > 0
not0test_idx = x_test.sum(1) > 0


In [122]:
train_data = train_data.loc[not0train_idx].reset_index(drop = True)
val_data = val_data.loc[not0val_idx].reset_index(drop = True)
test_data = test_data.loc[not0test_idx].reset_index(drop = True)

In [123]:
x_train = x_train[not0train_idx]
y_train = y_train[not0train_idx]

x_val = x_val[not0val_idx]
y_val = y_val[not0val_idx]

x_test = x_test[not0test_idx]
y_test = y_test[not0test_idx] 

In [124]:
not0test_idx.shape
# y_test = y_test[not0test_idx]에서 IndexError: boolean index did not match indexed array along dimension 0; dimension is 30000 but corresponding boolean dimension is 50000 에러가 났다.
    # 알고 보니 label으로 넘김 이 부분에서 test를 넣어야 하는데 val을 넣어서 발생한 오류이다

(50000,)

# Q6. 모델링 하시오.

**규칙**
* compile까지 마치시오.
* 모델 구조는 아래와 같다.
    1. Embedding layer : 임베딩 차원 192
    2. Conv1D : 필터수 128개, 윈도우 사이즈 5, swish
    3. Conv1D : 필터수 128개, 윈도우 사이즈 5, swish
    4. MaxPool1D : 필터사이즈 2
    5. Bidirectional layer :
        * 정방향 : LSTM, 48
        * 역방향 : LSTM, 48
    6. Bidirectional layer :
        * 정방향 : LSTM, 64
        * 역방향 : GRU, 64
    7. MaxPool1D : 필터사이즈 2
    7. Bidirectional layer :
        * 정방향 : GRU, 96
        * 역방향 : LSTM, 96
    8. Conv1D : 128개, 윈도우 사이즈 6, swish
    9. Flatten
    10. Fully connected, 노드 1024개, swish
    10. 적절한 아웃풋레이어.

In [127]:
####################
## Your Code here ##
####################
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Conv1D, Bidirectional, LSTM, GRU, RNN, MaxPool1D, Flatten


In [128]:
####################
## Your Code here ##
####################
# 1. 세션 초기화
clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape=(40,))

################################################################################
# 1. 임베딩 레이어 : 임베딩차원은 192
el = Embedding(max_words,
               embedding_dim,
               input_length=max_len)(il)

# 2. Conv1D 블록 : 필터수 128개, 윈도우 사이즈 5
hl1 = Conv1D(filters=128,
             kernel_size=5,
             activation='swish')(el)

# 3. Conv1D 블록 : 필터수 128개, 윈도우 사이즈 5
hl2 = Conv1D(filters=128,
             kernel_size=5,
             activation='swish')(hl1)

# 4. MaxPool1D 블록 : 필터사이즈2
hl3 = MaxPool1D(pool_size=2)(hl2)

# 5. Bidirectional layer :
#     * 정방향 : LSTM, 히든스테이트 48
#     * 역방향 : LSTM, 히든스테이트 48
forward_lstm48 = LSTM(48, return_sequences=True)
backward_lstm48 = LSTM(48, return_sequences=True, go_backwards=True)
hl4 = Bidirectional(forward_lstm48, backward_layer=backward_lstm48)(hl3)

# 6. Bidirectional layer :
#     * 정방향 : LSTM, 히든스테이트 64
#     * 역방향 : GRU, 히든스테이트 64
forward_lstm64 = LSTM(48, return_sequences=True)
backward_gru64 = GRU(64, return_sequences=True, go_backwards=True)
hl5 = Bidirectional(forward_lstm64, backward_layer=backward_gru64)(hl4)

# 7. MaxPool1D 블록 : 필터사이즈2
hl6 = MaxPool1D(pool_size=2)(hl5)

# 8. Bidirectional layer :
#     * 정방향 : GRU, 히든스테이트 96
#     * 역방향 : LSTM, 히든스테이트 96
forward_gru96 = GRU(96, return_sequences = True)
backward_lstm96 = LSTM(96, return_sequences = True, go_backwards = True)
hl7 = Bidirectional(forward_gru96, backward_layer = backward_lstm96)(hl6)

# 9. Conv1D 블록 : 필터수 128개, 윈도우 사이즈 6
hl8 = Conv1D(filters=128,
             kernel_size=6,
             activation='swish')(hl7)

# 10. 플래튼
hl9 = Flatten()(hl8)

# 11. FC Layer : 노드 1024개, swish
hl10 = Dense(1024, activation='swish')(hl9)

# 12. 적절한 아웃풋레이어
ol = Dense(1, activation='relu')(hl10)
################################################################################

# 3. 모델 처음과 끝 지정
model = Model(il, ol)

# 4. 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics='accuracy')

# 요약
model.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40)]              0         
                                                                 
 embedding (Embedding)       (None, 40, 192)           7680000   
                                                                 
 conv1d (Conv1D)             (None, 36, 128)           123008    
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 128)           82048     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 128)          0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 16, 96)           67968     
 l)                                                          

# Q7. Early Stopping을 이용하여 모델을 학습시키시오.

**규칙**
1. 트레이닝셋의 15%는 validation set으로 둔다.
2. batch size는 128 ~ 512 사이로 결정한다.
3. 10000 epochs
2. 10 epochs 이내에 개선이 안되면 stopping
3. 가장 성능이 좋았을 때의 가중치로 복원할 것


In [129]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=10,
                   verbose=1,
                   restore_best_weights=True)

model.fit(x_train, y_train, 
            epochs=10000, 
            batch_size = 128, 
            verbose=1, 
            callbacks=[es], 
            validation_split=0.15)

Epoch 1/10000
752/752 [==============================] - 417s 534ms/step - loss: 0.6410 - accuracy: 0.6816 - val_loss: 0.6267 - val_accuracy: 0.6905
Epoch 2/10000
752/752 [==============================] - 418s 556ms/step - loss: 7.4692 - accuracy: 0.5096 - val_loss: 7.5299 - val_accuracy: 0.5118
Epoch 3/10000
752/752 [==============================] - 420s 559ms/step - loss: 7.6908 - accuracy: 0.5014 - val_loss: 7.5299 - val_accuracy: 0.5118
Epoch 4/10000
752/752 [==============================] - 408s 543ms/step - loss: 7.6908 - accuracy: 0.5014 - val_loss: 7.5299 - val_accuracy: 0.5118
Epoch 5/10000
752/752 [==============================] - 408s 543ms/step - loss: 7.6908 - accuracy: 0.5014 - val_loss: 7.5299 - val_accuracy: 0.5118
Epoch 6/10000
752/752 [==============================] - 415s 552ms/step - loss: 7.6908 - accuracy: 0.5014 - val_loss: 7.5299 - val_accuracy: 0.5118
Epoch 7/10000
752/752 [==============================] - 409s 544ms/step - loss: 7.6908 - accuracy: 0.5014

# Q8. 테스트 셋위에서 평가를 하시오.

0. positive : 1, negative : 0
1. positive로 간주하는 기준을 [ 5%, 10%, ... , 95% ] 19개로 가져간다.
    * 예시) 두번째 기준은 positive할 확률이 10%만 넘어가도 postive하다고 간주한다.
2. 각 19개 기준에 대하여, model의 sensitivity 변화를 출력한다.
    * optional) 시각화를 하여도 좋다.
3. 각 19개 기준에 대하여, model의 'positive' class의 precision 변화를 출력한다.
    * optional) 시각화를 하여도 좋다.


In [ ]:
####################
## Your Code here ##
####################





# Q9. 다음 텍스트가 positive할 확률을 출력하시오.

"이 영화는 명작인지, 망작인지 너무나 혼란스럽고 즐겁다."




In [ ]:
####################
## Your Code here ##
####################





# Q10. 아래 주어진 문장을, 주어진 조건에 따라 분석하시오.

review = '명작이지ㅋ 특히 명절때만 되면 티비에서 참 많이 해줬었는데, 웃기기도하고, 액션도 가미되었고...특히 가족같은 형제라는 주제가 안방에 잘 먹혔던거 같다.'

* 루프문과 print를 이용하여 아래와 같은 방식으로 출력하시오.
    1. positive확률 :  50%,  인풋 : '명작이지ㅋ'
    2. positive확률 :  55%, 인풋 : '명작이지ㅋ 특히'
    3. positive확률 : 56%, 인풋 : '명작이지ㅋ 특히 명절때만'
    4. .....(계속 한 단어씩 증가 시키기)
    5. positive확률 : 98%, 인풋 : '명작이지~ 같다.'
* 인풋으로 사용한 문장과 확률은 명확히 출력되어야 한다.
    * 나머지는 출력되지 않아도 좋다.

In [ ]:
####################
## Your code here ##
####################




